# Analysis of *Kiki’s Delivery Service*

In [ ]:
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Set font as Arial
plt.rcParams['font.family'] = 'Arial'